# Reqs

In [2]:
# !pip install comet_ml

In [3]:
import pandas as pd
import numpy as np
import time
import gc
import pickle

from collections import Counter
from tqdm import tqdm
from sklearn.metrics import f1_score, confusion_matrix
from sklearn.model_selection import train_test_split
from typing import Any, Dict, Union, List, Optional, Tuple
from IPython.display import Audio

import warnings
warnings.filterwarnings("ignore")

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.nn import BCEWithLogitsLoss, CrossEntropyLoss, MSELoss

from transformers import Wav2Vec2Model, Wav2Vec2Config

from comet_ml import Experiment, init
from comet_ml.integration.pytorch import log_model, watch

from utils import seed_everything, empty_cache

device = torch.device('cuda' if torch.cuda.is_available()  else 'cpu')

In [4]:
# !gdown https://drive.google.com/drive/folders/1eGRZFSyvzbLLiY_zSmZJNuxwbzOWVZPe -O data --folder

In [5]:
base_dir = ".."
path_to_data_folder = f'{base_dir}/data_processed/words'
weights_folder = f"{base_dir}/weights"

train = pd.read_parquet(f'{path_to_data_folder}/train.parquet')
val = pd.read_parquet(f'{path_to_data_folder}/val.parquet')
test = pd.read_parquet(f'{path_to_data_folder}/test.parquet')

train

,file,text,start,end,confidence,label,data_type,subset,р_count,г_count,path
0,611b27e7-0019-4fc6-9622-21d9647c45f0.mp3,нет,0.00,0.60,0.504,1.0,first_stage_train,train,0,0,data_wav/train/611b27e7-0019-4fc6-9622-21d9647...
1,611b27e7-0019-4fc6-9622-21d9647c45f0.mp3,такого,0.80,1.04,0.990,1.0,first_stage_train,train,0,1,data_wav/train/611b27e7-0019-4fc6-9622-21d9647...
2,611b27e7-0019-4fc6-9622-21d9647c45f0.mp3,планируется,1.32,1.88,0.971,1.0,first_stage_train,train,1,0,data_wav/train/611b27e7-0019-4fc6-9622-21d9647...
3,67465147-b88c-4acd-bb91-a78340a9bde7.mp3,посмотреть,0.00,1.06,0.728,0.0,first_stage_train,train,1,0,data_wav/train/67465147-b88c-4acd-bb91-a78340a...
4,67465147-b88c-4acd-bb91-a78340a9bde7.mp3,пока,1.06,1.24,0.644,0.0,first_stage_train,train,0,0,data_wav/train/67465147-b88c-4acd-bb91-a78340a...
...,...,...,...,...,...,...,...,...,...,...,...
111405,67ee2e50-8c8a-49d8-b8f8-75cdda336930.mp3,потом,11.12,12.40,0.994,1.0,final_train,train,0,0,data_wav/train/67ee2e50-8c8a-49d8-b8f8-75cdda3...
111406,67ee2e50-8c8a-49d8-b8f8-75cdda336930.mp3,оплату,12.40,13.18,0.944,1.0,final_train,train,0,0,data_wav/train/67ee2e50-8c8a-49d8-b8f8-75cdda3...
111407,67ee2e50-8c8a-49d8-b8f8-75cdda336930.mp3,выгодную,13.32,13.84,0.677,1.0,final_train,train,0,1,data_wav/train/67ee2e50-8c8a-49d8-b8f8-75cdda3...
111408,67ee2e50-8c8a-49d8-b8f8-75cdda336930.mp3,работу,13.84,14.08,0.999,1.0,final_train,train,1,0,data_wav/train/67ee2e50-8c8a-49d8-b8f8-75cdda3...


In [6]:
with open(f"{base_dir}/data_processed/target_letters.pkl", "rb") as f:
    target_letters = pickle.load(f)

In [7]:
%%time

# with open('data/train_arrays.pkl', 'rb') as f1, open('data/val_arrays.pkl', 'rb') as f2, open('data/final_test_arrays.pkl', 'rb') as f3:
with open(f'{path_to_data_folder}/train_arrays.pkl', 'rb') as f1, open(f'{path_to_data_folder}/val_arrays.pkl', 'rb') as f2,  \
                                                                    open(f'{path_to_data_folder}/test_arrays.pkl', 'rb') as f3:
    train_arrays = pickle.load(f1)
    val_arrays = pickle.load(f2)
    test_arrays = pickle.load(f3)

CPU times: total: 7.36 s
Wall time: 7.4 s


In [8]:
def compute_class_weights_sqrt(y, degree=0.5):
    n_classes = y.nunique()
    
    weights = len(y) / (n_classes * np.bincount(y).astype(np.float64))
    weights = weights ** degree
    
    return weights

In [9]:
train['label'].value_counts(normalize=True)

label
0.0    0.633530
1.0    0.335964
2.0    0.027213
3.0    0.003293
Name: proportion, dtype: float64

In [10]:
class cfg:

    model_type = "wav2vec"
    model_name = "jonatasgrosman/wav2vec2-large-xlsr-53-russian"
    weights_folder = weights_folder

    batch_size = 32 if torch.cuda.is_available() else 2

    max_length = 16000 * 5

    target_letters = target_letters
    letter_count_weights = {}
    letters_num_classes = {}

    disorders_class_weights = torch.tensor(compute_class_weights_sqrt(train['label'].dropna()),
                                 device=device, dtype=torch.float32)

    label_smoothing_pretrain = 0.0
    label_smoothing_train = 0.0

    linear_probing_frac = 0.1 # Часть первой эпохи, в течение которой все веса, кроме головы, замораживаются
    zero_epoch_evaluation_frac = 0.1 # На какой части данных оценивать модель перед обучением (в начале 0-й эпохи)

    head_dim = 256
    dropout = 0.25
    lr_pretrain = 1e-4
    lr_train = 1e-4
    num_epochs_pretrain = 10
    num_epochs_train = 10
    metric_computation_times_per_epoch_train = 4
    metric_computation_times_per_epoch_val = 1

    early_stopping_pretrain = 1
    early_stopping_train = 3

cfg.save_model_name = f'{cfg.model_type}'
cfg.save_model_name

'wav2vec'

# Dataset

In [12]:
def get_rare_classes(data, target_letters):
    rare_borders = {}
    for letter in target_letters:
        count_sizes = data[f"{letter}_count"].value_counts(normalize=True).to_dict()
        rare_classes = [i for i, size in count_sizes.items() if size < 0.01]
        if rare_classes:
            rare_borders[letter] = min(rare_classes) - 1
        else:
            rare_borders[letter] = max(count_sizes.keys())

    return rare_borders

In [13]:
rare_borders = get_rare_classes(train, target_letters=target_letters)
rare_borders

{'р': 2, 'г': 1}

In [14]:
letter_count_weights = {}
letters_num_classes = {}

for letter in target_letters:
    train[f"{letter}_count"] = train[f"{letter}_count"].apply(lambda x: min(x, rare_borders[letter]))
    val[f"{letter}_count"] = val[f"{letter}_count"].apply(lambda x: min(x, rare_borders[letter]))
    test[f"{letter}_count"] = test[f"{letter}_count"].apply(lambda x: min(x, rare_borders[letter]))

    letter_count_weights[letter] = torch.tensor(compute_class_weights_sqrt(train[f"{letter}_count"]),
                             device=device, dtype=torch.float32)
    letters_num_classes[letter] = train[f"{letter}_count"].nunique()

cfg.letter_count_weights = letter_count_weights
cfg.letters_num_classes = letters_num_classes

In [15]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, df, arrays, target_letters=target_letters):

        # self.scale_factor = 2**15 # На что делим при переводе аудио из int во float
        df = df.reset_index(drop=True)

        self.files = df['file'].values
        self.arrays = arrays # [array/self.scale_factor for array in arrays]
        self.texts = df['text'].values
        self.target_letters = target_letters
        for letter in target_letters:
            setattr(self, f"{letter}_count", df[f"{letter}_count"])
            
        self.labels = df['label'].values

        self.text_lengths = [len(text) for text in df['text']]


    def __len__(self):
        return len(self.arrays)

    def __getitem__(self, idx):


        file = self.files[idx]
        array = self.arrays[idx]
        text = self.texts[idx]
        label = self.labels[idx]# if self.labels is not None else None


        batch = {
            'file': file,
            'input_values': array,
            'text': text,
            'label': label,
            'text_length': self.text_lengths[idx],
        }

        for letter in target_letters:
            batch[f"{letter}_counts"] = getattr(self, f"{letter}_count")[idx]

        return batch

train.fillna({"label": -100}, inplace=True)

dataset_train = CustomDataset(train, train_arrays, target_letters=target_letters)
dataset_val = CustomDataset(val, val_arrays, target_letters=target_letters)

In [16]:
class DataCollator:
    def __init__(self, cfg):
        self.cfg = cfg
        self.max_length = cfg.max_length

    def pad_arrays(self, arrays):
        max_batch_length = max(len(array) for array in arrays)
    
        arrays = torch.stack([torch.cat([array, torch.zeros(max_batch_length - len(array))]) for array in arrays])
        return arrays

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:

        arrays = [torch.tensor(feature["input_values"][:self.max_length], dtype=torch.float32) for feature in features]
        arrays = self.pad_arrays(arrays)

        labels = [feature["label"] for feature in features]

        batch = {'input_values': arrays}

        batch["labels"] = torch.tensor(labels, dtype=torch.long) if not np.isnan(labels[0]) else None

        for letter in cfg.target_letters:
            batch[f"{letter}_counts"] = torch.tensor([feature[f"{letter}_counts"] for feature in features], dtype=torch.long)
            
        batch["files"] = [feature["file"] for feature in features]

        return batch


data_collator = DataCollator(cfg=cfg)

In [17]:
dataloader_train = DataLoader(dataset_train, batch_size=cfg.batch_size, collate_fn=data_collator, shuffle=True)
dataloader_val = DataLoader(dataset_val, batch_size=cfg.batch_size*2, collate_fn=data_collator, shuffle=False)

In [18]:
for b in tqdm(dataloader_train):
    break
b

  0%|                                                                                         | 0/3468 [00:00<?, ?it/s]


{'input_values': tensor([[-3.7964e-02, -3.3569e-02, -2.7802e-02,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 1.8127e-02,  1.2024e-02,  4.1809e-03,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 1.1200e-02,  1.3336e-02,  1.0254e-02,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         ...,
         [-9.1553e-04, -2.6245e-03, -3.3569e-03,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [-3.0518e-04, -9.1553e-05,  2.1362e-04,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 7.8247e-02,  7.9315e-02,  7.7087e-02,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00]]),
 'labels': tensor([-100,    1,    0, -100, -100, -100,    0,    0,    1,    0,    1,    1,
            0, -100, -100,    0, -100,    0,    0,    0,    0,    0, -100,    0,
         -100,    0, -100,    1,    1, -100,    1, -100]),
 'р_counts': tensor([1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0,
         0, 0,

# Model

In [20]:
class DisordersDetector(nn.Module):

    def __init__(self, cfg, stage):
        super().__init__()
        self.cfg = cfg
        if stage == "pretrain":
            self.backbone = Wav2Vec2Model.from_pretrained(cfg.model_name)
        else:
            # Не подгружаем модель с HF, ведь мы подгрузим претрейн модель
            model_cfg = Wav2Vec2Config.from_pretrained(cfg.model_name)
            self.backbone = Wav2Vec2Model(model_cfg)
            
        self.stage = stage

        dropout = cfg.dropout
        hidden_dim = 1024
        head_dim = cfg.head_dim

        if stage == "pretrain":
            self.letter_count_heads = nn.ModuleDict({
                f"{letter}_count_head": nn.Sequential(
                    nn.Linear(hidden_dim, head_dim),
                    nn.Dropout(dropout),
                    nn.Linear(head_dim, cfg.letters_num_classes[letter])
                )
                for letter in cfg.target_letters
            })

            
            # for letter in cfg.target_letters:
            #     num_classes = cfg.letters_num_classes[letter]
            #     setattr(self, f"{letter}_count_head", nn.Sequential(nn.Linear(hidden_size, head_dim),
            #                                     nn.Dropout(dropout),
            #                                     nn.Linear(head_dim, num_classes)))
        else:
            self.disorders_head = nn.Sequential(nn.Linear(hidden_dim, head_dim),
                                            nn.Dropout(dropout),
                                            nn.Linear(head_dim, len(cfg.disorders_class_weights)))

    def forward(self, x):

        hidden_state = self.backbone(x).last_hidden_state
        pooled_output = torch.mean(hidden_state, dim=1)

        output = {}

        if self.stage == 'pretrain':
            # for letter in self.cfg.target_letters:
            #     head = getattr(self, f"{letter}_count_head")
            #     output[f'{letter}_count_output'] = self.g_count_head(pooled_output)
            for letter_head, head in self.letter_count_heads.items():
                output[letter_head[0]] = head(pooled_output)

        else:

            output['disorders'] = self.disorders_head(pooled_output)

        return output

    def freeze_feature_extractor(self):
        self.backbone.feature_extractor._freeze_parameters()

    def freeze_backbone(self):
        for param in self.backbone.parameters():
            param.requires_grad = False

    def unfreeze_backbone(self):
            for param in self.backbone.parameters():
                param.requires_grad = True

            if self.cfg.model_type == "wav2vec":
                self.freeze_feature_extractor()


# Functions for train

In [103]:
def get_metric_pretrain(all_predictions, all_targets, is_val=False):

    metrics = []
    
    for letter in all_predictions.keys():
        # print(all_predictions, letter)
        if len(all_predictions[letter]) == 0: continue
        predictions = np.array(all_predictions[letter]).argmax(axis=-1)
        targets = np.array(all_targets[letter])

        metric = f1_score(targets, predictions, average='macro')
        metrics.append(metric)
        # metrics['letter'] = letter
        print(f"{letter} metric\n", metric)
        if is_val:
            print(confusion_matrix(targets, predictions))

    return metrics, np.mean(metrics)


def get_metric_train(predictions, targets):
    predictions = np.array(predictions['disorders']).argmax(axis=-1)
    targets = np.array(targets['disorders'])

    metric = f1_score(targets, predictions, average='macro')

    print(confusion_matrix(targets, predictions))

    return metric

In [104]:
def model_step(model, stage, batch, cfg, predictions, targets, all_files, criterions):
    loss = 0

    if stage == 'pretrain':

        output = model(batch['input_values'].to(model.backbone.device))

        for letter in cfg.target_letters:
            loss += criterions[f"{letter}_count_head"](output[letter], batch[f'{letter}_counts'].to(model.backbone.device))

            predictions[f'{letter}_count'].extend(F.softmax(output[letter], dim=-1).detach().cpu().numpy())
            targets[f'{letter}_count'].extend(batch[f'{letter}_counts'].cpu().numpy().flatten())

        all_files.extend(np.array(batch['files']))

    else:

        batch_targets = batch['labels'].to(model.backbone.device)

        output = model(batch['input_values'].to(model.backbone.device))

        loss += criterions['disorders'](output['disorders'], batch_targets)

        predictions['disorders'].extend(F.softmax(output['disorders'], dim=-1).detach().cpu().numpy())
        targets['disorders'].extend(batch_targets.cpu().numpy().flatten())

        all_files.extend(np.array(batch['files']))


    return loss

In [105]:
def train_model(model, cfg, dataloader_train, dataloader_val, stage='pretrain'):

    if stage == 'pretrain':
        optimizer = torch.optim.Adam(model.parameters(), lr=cfg.lr_pretrain)
        num_epochs = cfg.num_epochs_pretrain
    else:
        optimizer = torch.optim.Adam(model.parameters(), lr=cfg.lr_train)
        num_epochs = cfg.num_epochs_train

    scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)


    best_metric = -1
    best_epoch = -1

    if cfg.linear_probing_frac > 0.0:
        # Замораживаем тело в начале обучения
        model.freeze_backbone()

    num_batches = len(dataloader_train)
    unfreeze_backbone_batch = int(num_batches * min(cfg.linear_probing_frac, 1.0))

    criterions = {}
    if stage == 'pretrain':
        save_weights_name = f"{cfg.save_model_name}-pretrain"
        for letter in cfg.target_letters:
            criterions[f"{letter}_count_head"] = torch.nn.CrossEntropyLoss(weight=cfg.letter_count_weights[letter],
                                                      label_smoothing=cfg.label_smoothing_pretrain)

    elif stage == 'train':

        save_weights_name = f"{cfg.save_model_name}-train"
        
        criterions['disorders'] = torch.nn.CrossEntropyLoss(weight=cfg.disorders_class_weights,
                                                   label_smoothing=cfg.label_smoothing_train, reduction='mean')
        
    for epoch in range(num_epochs):
        if stage == "pretrain" and epoch > best_epoch + cfg.early_stopping_pretrain: # Метрика не улучшается долго
            break
        elif stage == "train" and epoch > best_epoch + cfg.early_stopping_train: # Метрика не улучшается долго
            break

        print('*'*50)
        print('EPOCH', epoch)
        print('*'*50)

        model.train()
        
        predictions = {'disorders': []}
        targets = {'disorders': []}

        for letter in cfg.target_letters:
            predictions[f'{letter}_count'] = []
            targets[f'{letter}_count'] = []
            

        all_files = []

        for batch_idx, batch in enumerate(tqdm(dataloader_train, total=len(dataloader_train), desc='Training')):


            if sanity_checking and batch_idx  >= 2: break
            if epoch == 0 and cfg.linear_probing_frac > 0.0 and batch_idx == unfreeze_backbone_batch:
                model.unfreeze_backbone()

            optimizer.zero_grad()

            loss = model_step(model, stage, batch, cfg, predictions,
                                              targets, all_files,
                                              criterions)

            experiment.log_metric("loss", loss, step=batch_idx + (num_batches * epoch))

            del batch
            torch.cuda.empty_cache()

            loss.backward()
            optimizer.step()


            if batch_idx > 0 and batch_idx % max(1, len(dataloader_train) // cfg.metric_computation_times_per_epoch_train) == 0:
                
                if stage == 'pretrain':
                    metrics, mean_metric = get_metric_pretrain(predictions, targets, is_val=False)
                    for letter, metric in zip(cfg.target_letters, metrics):
                        experiment.log_metric(f"f1_{letter}_count", metric, step=batch_idx + (num_batches * epoch))
                        
                    experiment.log_metric("f1_mean", mean_metric, step=batch_idx + (num_batches * epoch))
                    metric = mean_metric
                else:
                    metric = get_metric_train(predictions, targets)
                    experiment.log_metric("f1_disorders", metric, step=batch_idx + (num_batches * epoch))

                print(metric)


            
            if (batch_idx == 0 and epoch == 0):
                metric = evaluate(model, cfg, dataloader_val, criterions, stage=stage, is_beggining=True, step_idx=batch_idx + (num_batches*epoch))
                model.train()

            elif (batch_idx + 1) % (len(dataloader_train) // cfg.metric_computation_times_per_epoch_val) == 0:
                metric = evaluate(model, cfg, dataloader_val, criterions, stage=stage, step_idx=batch_idx + (num_batches*epoch))
                model.train()
                
                if metric > best_metric:
                    best_metric = metric
                    best_epoch = epoch
                    torch.save(model.state_dict(), f'{cfg.weights_folder}/{save_weights_name}.pt')


        if sanity_checking:
            break

        scheduler.step()


In [106]:
def evaluate(model, cfg, dataloader_val, criterions, is_beggining=False, limit_num_batches=-1, stage='pretrain', step_idx=0):



    num_batches = len(dataloader_val)
    zero_epoch_evaluation_batches = np.ceil(num_batches * min(cfg.zero_epoch_evaluation_frac, 1.0))

    model.eval()

    predictions = {'disorders': []}
    targets = {'disorders': []}

    for letter in cfg.target_letters:
        predictions[f'{letter}_count'] = []
        targets[f'{letter}_count'] = []

    all_files = []

    with torch.no_grad():
        for batch_idx, batch in enumerate(dataloader_val):
            if sanity_checking and batch_idx  == 5: break
            if batch_idx == limit_num_batches: break # Оценивал по 50 батчам, т.к. иначе времени бы много занимало, можно исправить и оценивать по фулл даталоадеру, но раз в эпоху
            if is_beggining and batch_idx == zero_epoch_evaluation_batches: break

            loss = model_step(model, stage, batch, cfg, 
                              predictions, targets, 
                              all_files, criterions)


            del batch
            torch.cuda.empty_cache()

    print('\n VALIDATION')
    print('=*'*50)
    
    if stage == 'pretrain':
        metrics, mean_metric = get_metric_pretrain(predictions, targets, is_val=True)
        
        for letter, metric in zip(cfg.target_letters, metrics):
            experiment.log_metric(f"val_f1_{letter}_count", metric, step=step_idx)
            
        experiment.log_metric("val_f1_mean", mean_metric, step=step_idx)

        metric = mean_metric
    else:
        metric = get_metric_train(predictions, targets)
        experiment.log_metric("val_f1_disorders", metric, step=step_idx)

    print(metric)
    print('=*'*50)

    return metric

In [107]:
sanity_checking = False

# PreTraining

In [28]:
experiment = Experiment(
  api_key="rpI0PuxxYkKMtiy42g1oIfLI1",
  project_name="aiijc-final-pretrain",
  workspace="ugryumnik"
)

experiment.log_parameters(dict(vars(cfg)))

COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: torch, sklearn.
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/ugryumnik/aiijc-final-pretrain/312cb9acf6a942fd810d67ca5b2acf82

COMET INFO: Couldn't find a Git repository in 'C:\\Users\\User\\jupyter_lab\\aiijc_tbank\\train_notebooks' nor in any parent directory. Set `COMET_GIT_DIRECTORY` if your Git Repository is elsewhere.


In [29]:
empty_cache()

seed_everything(42)

model = DisordersDetector(cfg=cfg, stage='pretrain')
model.to(device)
if cfg.model_type == "wav2vec":
    model.freeze_feature_extractor()

In [30]:
len(dataset_train)

110974

In [31]:
train_model(model, cfg, dataloader_train, dataloader_val, stage='pretrain')

**************************************************
EPOCH 0
**************************************************


Training:   0%|                                                                     | 2/3468 [00:02<1:04:00,  1.11s/it]


 VALIDATION
=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*
р_count metric
 0.30920009822963407
[[236   6   0]
 [ 69   3   0]
 [  6   0   0]]
г_count metric
 0.477124183006536
[[292   0]
 [ 28   0]]
0.393162140618085
=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*


Training:  25%|█████████████████▎                                                   | 868/3468 [04:08<20:47,  2.08it/s]

р_count metric
 0.4062452101035658
г_count metric
 0.5063225053557862
0.45628385772967595


...

р_count metric
 0.8019005456650415
г_count metric
 0.8342780442239921
0.8180892949445169


Training:  75%|███████████████████████████████████████████████████                 | 2602/3468 [16:42<07:18,  1.98it/s]

р_count metric
 0.8028603798906789
г_count metric
 0.8346591990638679
0.8187597894772733


Training: 100%|████████████████████████████████████████████████████████████████████| 3468/3468 [23:01<00:00,  2.51it/s]


 VALIDATION
=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*
р_count metric
 0.8173635275414481
[[1897  134   11]
 [  85  661   29]
 [   5    5   51]]
г_count metric
 0.8339517131191194
[[2542   82]
 [  74  180]]
0.8256576203302838
=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*


In [32]:
log_model(experiment, model, "pretrain-stage")
experiment.end()

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : worthy_mammal_343
COMET INFO:     url                   : https://www.comet.com/ugryumnik/aiijc-final-pretrain/312cb9acf6a942fd810d67ca5b2acf82
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     f1_mean [15]       : (0.45628385772967595, 0.8205152964466003)
COMET INFO:     f1_г_count [15]    : (0.5063225053557862, 0.8346591990638679)
COMET INFO:     f1_р_count [15]    : (0.4062452101035658, 0.8107262261233833)
COMET INFO:     loss [17340]       : (0.1022215187549591, 2.500896453857422)
COMET INFO:     val_f1_mean [6]    : (0.393162140618085, 0.8385460821164437)
COMET INFO:     val_f1_г_count [6] : (0.477124183006536, 0.8339517131191194)
COMET 

# Training

## Data Cleaning
Немного очистим от ненужных для обучения данных

In [35]:
train = train.reset_index(drop=True)
train.shape

(110974, 11)

In [36]:
train = train[train['label'] != -100] # Удаляем данные, для которых нет таргета
train.shape

(82607, 11)

In [37]:
classes_count = train['label'].value_counts(normalize=True)
rare_classes = classes_count[classes_count < 0.05].index.values
rare_classes

array([2., 3.])

In [38]:
target_letters_count = train[[f"{letter}_count" for letter in target_letters]].sum(axis=1)


ids_to_drop = train[(target_letters_count == 0) & (~train['label'].isin(rare_classes))].index
ids_to_drop, _ = train_test_split(ids_to_drop, test_size=0.05, random_state=42)


train.drop(index=ids_to_drop, inplace=True)
train.shape

(31575, 11)

In [39]:
train_arrays = [train_arrays[i] for i in train.index]
train.reset_index(drop=True, inplace=True)
assert train.shape[0] == len(train_arrays)

In [40]:
dataset_train = CustomDataset(train, train_arrays, target_letters=target_letters)
dataset_val = CustomDataset(val, val_arrays, target_letters=target_letters)
dataset_test = CustomDataset(test, test_arrays, target_letters=target_letters)


dataloader_train = DataLoader(dataset_train, batch_size=cfg.batch_size, collate_fn=data_collator, shuffle=True)
dataloader_val = DataLoader(dataset_val, batch_size=cfg.batch_size*2, collate_fn=data_collator, shuffle=False)
dataloader_test = DataLoader(dataset_test, batch_size=cfg.batch_size*2, collate_fn=data_collator, shuffle=False)

## Теперь обучим модель

In [42]:
experiment = Experiment(
  api_key="rpI0PuxxYkKMtiy42g1oIfLI1",
  project_name="aiijc-final-train",
  workspace="ugryumnik"
)

experiment.log_parameters(dict(vars(cfg)))

COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: torch, sklearn.
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/ugryumnik/aiijc-final-train/c8ed27196b344832adb5dc7c854b785f

COMET INFO: Couldn't find a Git repository in 'C:\\Users\\User\\jupyter_lab\\aiijc_tbank\\train_notebooks' nor in any parent directory. Set `COMET_GIT_DIRECTORY` if your Git Repository is elsewhere.


In [130]:
empty_cache()

seed_everything(42)

dataloader_train = DataLoader(dataset_train, batch_size=cfg.batch_size, collate_fn=data_collator, shuffle=True)
dataloader_val = DataLoader(dataset_val, batch_size=cfg.batch_size*2, collate_fn=data_collator, shuffle=False)

model = DisordersDetector(cfg=cfg, stage='train')
model.to(device)

current_model_dict = model.state_dict()
loaded_state_dict = torch.load(f"{cfg.weights_folder}/{cfg.save_model_name}-pretrain.pt", map_location=device, weights_only=True)

for k in current_model_dict.keys():
    if k in loaded_state_dict.keys():
        current_model_dict[k] = loaded_state_dict[k]

del loaded_state_dict
empty_cache()

model.load_state_dict(current_model_dict, strict=False)

if cfg.model_type == "wav2vec":
    model.freeze_feature_extractor()

In [131]:
train_model(model, cfg, dataloader_train, dataloader_val, stage='train')

**************************************************
EPOCH 0
**************************************************


Training:   0%|                                                                        | 1/987 [00:02<38:12,  2.33s/it]


 VALIDATION
=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*
[[155  18   0  20]
 [ 87  10   0  11]
 [ 13   3   0   3]
 [  0   0   0   0]]
0.20896229445015416
=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*


Training:  25%|█████████████████▌                                                    | 247/987 [01:15<04:48,  2.57it/s]

[[4198   77  409   57]
 [2249   39  200   36]
 [ 358    4  181    6]
 [  53    3   34    0]]
0.25424476338013746


...

Training: 100%|█████████████████████████████████████████████████████████████████████▊| 985/987 [06:38<00:00,  2.60it/s]

[[15618  2690   613    44]
 [ 2812  6971   220    38]
 [  145    35  2052    10]
 [   10     4     5   253]]
0.7894705562887242


Training: 100%|█████████████████████████████████████████████████████████████████████▉| 986/987 [06:38<00:00,  2.81it/s]


 VALIDATION
=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*
[[1083  359  301   16]
 [ 384  309  177    7]
 [  64   34   90    2]
 [  21   19    8    4]]
0.3436672050788729
=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*


Training: 100%|██████████████████████████████████████████████████████████████████████| 987/987 [06:56<00:00,  2.37it/s]


In [142]:
log_model(experiment, model, "train-stage")
experiment.end()

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : distant_barracuda_917
COMET INFO:     url                   : https://www.comet.com/ugryumnik/aiijc-final-train/c8ed27196b344832adb5dc7c854b785f
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     f1_disorders [40]     : (0.25424476338013746, 0.7894705562887242)
COMET INFO:     loss [10611]          : (0.09200619906187057, 2.7596871852874756)
COMET INFO:     val_f1_disorders [14] : (0.20896229445015416, 0.3436672050788729)
COMET INFO:   Others:
COMET INFO:     hasNestedParams : True
COMET INFO:   Parameters:
COMET INFO:     __dict__                                 : <attribute '__dict__' of 'cfg' objects>
COMET INFO:     __doc__                

In [143]:
cfg_dict = {attr: getattr(cfg, attr) for attr in dir(cfg) if not callable(getattr(cfg, attr)) and not attr.startswith("__")}

with open(f"{base_dir}/data_processed/cfg.pkl", 'wb') as f:
    pickle.dump(cfg_dict, f)

# Inference

In [144]:
empty_cache()

model = DisordersDetector(cfg=cfg, stage='train')
model.to(device)

model.load_state_dict(torch.load(f"{cfg.weights_folder}/{cfg.save_model_name}-train.pt", map_location=device, weights_only=True), strict=False)
model = model.eval()

In [145]:
def get_predictions(model, dataloader):
    model.eval()
    all_predictions = []

    with torch.no_grad():
        for batch_idx, batch in tqdm(enumerate(dataloader), total=len(dataloader)):

            output = model(batch['input_values'].to(device))

            all_predictions.extend(F.softmax(output['disorders'], dim=-1).cpu().numpy())
            
            del batch
            torch.cuda.empty_cache()

    return all_predictions

In [146]:
all_predictions_val = get_predictions(model, dataloader_val)

with open(f"{base_dir}/preds/all_predictions_val.pkl", "wb") as f:
    pickle.dump(all_predictions_val, f)

100%|██████████████████████████████████████████████████████████████████████████████████| 45/45 [00:17<00:00,  2.63it/s]


In [147]:
all_predictions_test = get_predictions(model, dataloader_test)

with open(f"{base_dir}/preds/all_predictions_test.pkl", "wb") as f:
    pickle.dump(all_predictions_test, f)

100%|██████████████████████████████████████████████████████████████████████████████████| 72/72 [00:27<00:00,  2.59it/s]
